<a href="https://colab.research.google.com/github/YIKUAN8/Transformers-VQA/blob/master/openI_VQA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**In this notebook, we will classify 15 thoracic findings from Chest X-ray images and associated reports. This can be considered as an VQA task. We will fine-tune 3 pre-trained transformer based V+L models. After running through this notebook, you will be able to fine-tune these models on your customized dataset.**

####**0.1 clone our repo and install dependencies!**


In [ ]:
!git clone https://github.com/YIKUAN8/Transformers-VQA.git
%cd Transformers-VQA/
!pip install -r requirements.txt


**Change the 79th line of param.py from**
```
args = parser.parse_args()
```
to
```
args = parser.parse_args([])
```
This will enable us to use *argparse* in jupyter notebook!



#### **0.2 Download pre-trained models and place them to data/pretrained/, you could choose from [VisualBERT](https://github.com/uclanlp/visualbert), [LXMERT](https://github.com/airsplay/lxmert), [UNITER](https://github.com/ChenRocks/UNITER).**

In [3]:
#line 1: UNITER; line 2:LXMERT, line 3: VisualBERT. Comment out selected lines if you don't want to use this model
#if the pre-trained VisualBERT cannot be downloaded succesfully, rerun one more time or refer to this link: https://drive.google.com/file/d/1kuPr187zWxSJbtCbVW87XzInXltM-i9Y/view?usp=sharing
!wget https://convaisharables.blob.core.windows.net/uniter/pretrained/uniter-base.pt -P models/pretrained/
!wget --no-check-certificate https://nlp1.cs.unc.edu/data/model_LXRT.pth -P models/pretrained/
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y" -O models/pretrained/visualbert.th && rm -rf /tmp/cookies.txt


uniter-base.pt      100%[===================>] 260.93M   870KB/s    in 5m 32s  

2023-04-10 16:58:29 (804 KB/s) - 'models/pretrained/uniter-base.pt' saved [273600756/273600756]

--2023-04-10 16:58:29--  https://nlp1.cs.unc.edu/data/model_LXRT.pth
Resolving nlp1.cs.unc.edu (nlp1.cs.unc.edu)... 152.2.133.65
Connecting to nlp1.cs.unc.edu (nlp1.cs.unc.edu)|152.2.133.65|:443... failed: Connection timed out.
Retrying.

--2023-04-10 17:00:40--  (try: 2)  https://nlp1.cs.unc.edu/data/model_LXRT.pth
Connecting to nlp1.cs.unc.edu (nlp1.cs.unc.edu)|152.2.133.65|:443... ^C
--2023-04-10 17:02:47--  https://docs.google.com/uc?export=download&confirm=t&id=1kuPr187zWxSJbtCbVW87XzInXltM-i9Y
Resolving docs.google.com (docs.google.com)... 142.250.4.101, 142.250.4.100, 142.250.4.139, ...
Connecting to docs.google.com (docs.google.com)|142.250.4.101|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0g-5c-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7de

#### **0.3 Download OpenI dataset.**

A detailed description of this dataset can be found at [here](https://openi.nlm.nih.gov/). In summary, there are 3684 CXR Image-Report pairs in this dataset. Each pair has an annotation of 15 throacic findings from MESH terms. We convert the raw data to a dataframe with better visibility. It can be accessed with the following command or this [link](https://drive.google.com/file/d/1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG/view?usp=sharing).

In [4]:
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG" -O data/openIdf.csv && rm -rf /tmp/cookies.txt


--2023-04-10 17:05:16--  https://docs.google.com/uc?export=download&confirm=&id=1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG
Resolving docs.google.com (docs.google.com)... 142.251.12.113, 142.251.12.138, 142.251.12.101, ...
Connecting to docs.google.com (docs.google.com)|142.251.12.113|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-14-4s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8as8dtbaq416ipq8jjj91ib0ajdok67c/1681146300000/09550986323973647809/*/1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG?e=download&uuid=ac25827c-dafe-4984-b437-52eb20593e90 [following]
--2023-04-10 17:05:17--  https://doc-14-4s-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/8as8dtbaq416ipq8jjj91ib0ajdok67c/1681146300000/09550986323973647809/*/1i3wcfXJbH_4q3rS2rvLxtzbMiO-KuZCG?e=download&uuid=ac25827c-dafe-4984-b437-52eb20593e90
Resolving doc-14-4s-docs.googleusercontent.com (doc-14-4s-docs.googleusercontent.com)... 74.125.68.132, 24

***0.3.1 Have a glance of this dataframe, column 'TXT' is the radiology report; column 'split' and 'id' are self-explantory; All other columns are the 15 findings. Our task will be a 15-labels binary classification with visual and semantic input.***

In [296]:
import pandas as pd
import os

os.chdir('/home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA')

openI = pd.read_csv('data/openIdf.csv',index_col=0)
openI.shape
openI

,id,Atelectasis,Cardiomegaly,Effusion,Infiltration,Mass,Nodule,Pneumonia,Pneumothorax,Consolidation,Edema,Emphysema,Fibrosis,Pleural_Thickening,Hernia,Normal,split,TXT
0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,The cardiac silhouette and mediastinum size ar...
1021,2,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,train,Borderline cardiomegaly. Midline sternotomy XX...
2020,3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,"No displaced rib fractures, pneumothorax, or ..."
3061,4,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,train,There are diffuse bilateral interstitial and a...
3169,5,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,train,The cardiomediastinal silhouette and pulmonary...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3056,3995,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,train,The cardiomediastinal silhouette and pulmonary...
3057,3996,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,The lungs are clear. Heart size is normal. No ...
3058,3997,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,"Heart size within normal limits. Small, nodula..."
3059,3998,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,train,Heart size is normal and the lungs are clear.


In [297]:
data_ct = pd.read_csv('https://raw.githubusercontent.com/AndiAlifs/Channelwise-Saab-Transform/main/data_ct.csv')
data_ct.head()

,filename,clinical_notes,modality,No Finding,Pneumonia,Pneumonia/Aspiration,Pneumonia/Bacterial,Pneumonia/Bacterial/Chlamydophila,Pneumonia/Bacterial/E.Coli,Pneumonia/Bacterial/Klebsiella,...,Pneumonia/Viral/COVID-19,Pneumonia/Viral/Herpes,Pneumonia/Viral/Influenza,Pneumonia/Viral/Influenza/H1N1,Pneumonia/Viral/MERS-CoV,Pneumonia/Viral/SARS,Pneumonia/Viral/Varicella,Tuberculosis,Unknown,todo
0,3ED3C0E1-4FE0-4238-8112-DDFF9E20B471.jpeg,"Standard CT, reconstruction with lung algorith...",CT,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
1,16745_7_4.png,Patient 2 - CT scan during hospitalization – a...,CT,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
2,16745_4_2.png,Patient 3 - CT scan – axial view: bilateral an...,CT,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
3,coronacases_org_009.nii.gz,CT images reveal ground-glass opacities on the...,CT,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False
4,16631_1_4.jpg,"A female patient, 39-years-old, fever (38.1℃) ...",CT,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,False


In [338]:
data_xray = pd.read_csv('https://raw.githubusercontent.com/AndiAlifs/Channelwise-Saab-Transform/main/data_xray.csv')

# set filename as index
data_xray.set_index('filename', inplace=True)
data_xray.drop(columns=['modality'], inplace=True)
# change clinical notes column name to TXT
data_xray.rename(columns={'clinical_notes':'TXT'}, inplace=True)

# add id with number from 1
data_xray['id'] = range(1, len(data_xray) + 1)

# add 'split' column where 20% contain 'test' and 80% contain 'train'
data_xray['split'] = 'train'
data_xray.loc[data_xray.sample(frac=0.2, random_state=42).index, 'split'] = 'test'

# move split column to second from last column
colname = ['id', 'todo', 'No Finding', 'Pneumonia', 'Pneumonia/Aspiration', 'Pneumonia/Bacterial', 'Pneumonia/Bacterial/Chlamydophila', 'Pneumonia/Bacterial/E.Coli', 'Pneumonia/Bacterial/Klebsiella', 'Pneumonia/Bacterial/Legionella', 'Pneumonia/Bacterial/Mycoplasma', 'Pneumonia/Bacterial/Nocardia', 'Pneumonia/Bacterial/Staphylococcus/MRSA', 'Pneumonia/Bacterial/Streptococcus', 'Pneumonia/Fungal/Aspergillosis', 'Pneumonia/Fungal/Pneumocystis', 'Pneumonia/Lipoid', 'Pneumonia/Viral/COVID-19', 'Pneumonia/Viral/Herpes ', 'Pneumonia/Viral/Influenza', 'Pneumonia/Viral/Influenza/H1N1', 'Pneumonia/Viral/MERS-CoV', 'Pneumonia/Viral/SARS', 'Pneumonia/Viral/Varicella', 'Tuberculosis', 'Unknown','split','TXT']
data_xray = data_xray[colname]

# change false to 0 and true to 1
data_xray = data_xray.replace(False, 0)
data_xray = data_xray.replace(True, 1)

# change to int
data_xray = data_xray.astype({'id': 'int32', 'todo': 'int32', 'No Finding': 'int32', 'Pneumonia': 'int32', 'Pneumonia/Aspiration': 'int32', 'Pneumonia/Bacterial': 'int32', 'Pneumonia/Bacterial/Chlamydophila': 'int32', 'Pneumonia/Bacterial/E.Coli': 'int32', 'Pneumonia/Bacterial/Klebsiella': 'int32', 'Pneumonia/Bacterial/Legionella': 'int32', 'Pneumonia/Bacterial/Mycoplasma': 'int32', 'Pneumonia/Bacterial/Nocardia': 'int32', 'Pneumonia/Bacterial/Staphylococcus/MRSA': 'int32', 'Pneumonia/Bacterial/Streptococcus': 'int32', 'Pneumonia/Fungal/Aspergillosis': 'int32', 'Pneumonia/Fungal/Pneumocystis': 'int32', 'Pneumonia/Lipoid': 'int32', 'Pneumonia/Viral/COVID-19': 'int32', 'Pneumonia/Viral/Herpes ': 'int32', 'Pneumonia/Viral/Influenza': 'int32', 'Pneumonia/Viral/Influenza/H1N1': 'int32', 'Pneumonia/Viral/MERS-CoV': 'int32', 'Pneumonia/Viral/SARS': 'int32', 'Pneumonia/Viral/Varicella': 'int32', 'Tuberculosis': 'int32', 'Unknown': 'int32'})

data_xray.head()

,id,todo,No Finding,Pneumonia,Pneumonia/Aspiration,Pneumonia/Bacterial,Pneumonia/Bacterial/Chlamydophila,Pneumonia/Bacterial/E.Coli,Pneumonia/Bacterial/Klebsiella,Pneumonia/Bacterial/Legionella,...,Pneumonia/Viral/Herpes,Pneumonia/Viral/Influenza,Pneumonia/Viral/Influenza/H1N1,Pneumonia/Viral/MERS-CoV,Pneumonia/Viral/SARS,Pneumonia/Viral/Varicella,Tuberculosis,Unknown,split,TXT
filename,,,,,,,,,,,,,,,,,,,,,
auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,"On January 22, 2020, a 65-year-old man with a ..."
auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,"On January 22, 2020, a 65-year-old man with a ..."
auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,"On January 22, 2020, a 65-year-old man with a ..."
auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,"On January 22, 2020, a 65-year-old man with a ..."
nejmc2001573_f1a.jpeg,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,diffuse infiltrates in the bilateral lower lungs


In [340]:
data_xray['TXT'].fillna('Normal', inplace=True)

In [341]:
data_xray

,id,todo,No Finding,Pneumonia,Pneumonia/Aspiration,Pneumonia/Bacterial,Pneumonia/Bacterial/Chlamydophila,Pneumonia/Bacterial/E.Coli,Pneumonia/Bacterial/Klebsiella,Pneumonia/Bacterial/Legionella,...,Pneumonia/Viral/Herpes,Pneumonia/Viral/Influenza,Pneumonia/Viral/Influenza/H1N1,Pneumonia/Viral/MERS-CoV,Pneumonia/Viral/SARS,Pneumonia/Viral/Varicella,Tuberculosis,Unknown,split,TXT
filename,,,,,,,,,,,,,,,,,,,,,
auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,"On January 22, 2020, a 65-year-old man with a ..."
auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,"On January 22, 2020, a 65-year-old man with a ..."
auntminnie-c-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,"On January 22, 2020, a 65-year-old man with a ..."
auntminnie-d-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,"On January 22, 2020, a 65-year-old man with a ..."
nejmc2001573_f1a.jpeg,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,diffuse infiltrates in the bilateral lower lungs
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
072ecaf8c60a81980abb57150a8016_jumbo-9.jpeg,862,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,Asthmatic. Shortness of breath and wheeze. Rou...
ff33c406392b968d483174c97eb857_jumbo-9.jpeg,863,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,train,Asthmatic. Shortness of breath and wheeze. The...
000001-266.jpg,864,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,test,fire-eater accidentally ingested a paraffin mi...


In [299]:
# change TXT in first 866 data of openI to clinical_notes from data_xray
openI['TXT'][0:866] = data_xray['clinical_notes']

# replace nan in TXT withh "Normal" 
openI['TXT'] = openI['TXT'].fillna('Normal')

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [301]:
openI.isnull().sum()

id                    0
Atelectasis           0
Cardiomegaly          0
Effusion              0
Infiltration          0
Mass                  0
Nodule                0
Pneumonia             0
Pneumothorax          0
Consolidation         0
Edema                 0
Emphysema             0
Fibrosis              0
Pleural_Thickening    0
Hernia                0
Normal                0
split                 0
TXT                   0
dtype: int64

#### **0.4 Download the visaul features extracted by BUTD. 36 2048-dimension visual feature is extracted from each CXR Image. We use this [implementation](https://github.com/airsplay/py-bottom-up-attention). This step will take a while (~1min). To save downloading time, you can also make a copy of this [shareable link](https://drive.google.com/file/d/1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ/view?usp=sharing) to your own google drive and mount you colab to your gdrive.**


*If you are interested in the original CXR images, which is unnecessary to out project , you can access them [here](https://drive.google.com/drive/folders/1s5A0CFB6-2N5ThbuorUK1t-bUEKmZnjz?usp=sharing).*

In [ ]:
#!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1BFw0jc0j-ffT2PhI4CZeP3IJFZg3GxlZ" -O data/openI_v_features.pickle && rm -rf /tmp/cookies.txt

***0.4.1 Load visual features***

In [302]:
import pickle
openI_v_f = pickle.load( open( "/home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA/data/openI_v_features.pickle", "rb" ) )

In [303]:
openI_v_f[842]

[array([[4.39822205e+02, 2.09190521e+02, 5.05938446e+02, 4.84556580e+02],
        [4.54583191e+02, 1.91853088e+02, 5.10664551e+02, 3.85950684e+02],
        [3.55818451e+02, 1.91926041e+02, 5.08805115e+02, 5.07845428e+02],
        [2.88787766e+01, 7.69064713e+01, 2.07315842e+02, 4.21507050e+02],
        [7.31768799e+01, 1.11570641e+02, 4.44939331e+02, 5.09288025e+02],
        [7.46494818e+00, 0.00000000e+00, 4.85485840e+02, 3.20990906e+02],
        [0.00000000e+00, 1.21823738e+02, 2.89714325e+02, 5.11456787e+02],
        [4.62462555e+02, 3.01870575e+01, 5.11625488e+02, 4.94210846e+02],
        [2.74601593e+02, 7.67158890e+00, 5.10533966e+02, 4.51702606e+02],
        [1.20879524e+02, 1.22537987e+02, 3.58050476e+02, 4.11514740e+02],
        [4.91042519e+01, 2.21398697e+01, 2.14258942e+02, 2.93460205e+02],
        [1.71222214e+02, 8.81950378e+01, 4.09603851e+02, 3.51593384e+02],
        [3.13130402e+02, 7.53816910e+01, 4.82260223e+02, 3.79200714e+02],
        [1.68782120e+02, 3.29532051e+0

In [304]:
assert set(list(openI_v_f.keys())) == set(openI.id.values), "Visual Features are inconsistent with openI dataset"

In [305]:
len(set(list(openI_v_f.keys())))

3684

In [306]:
list(openI_v_f[1])

[array([[0.00000000e+00, 6.88257141e+01, 3.15785339e+02, 4.00191406e+02],
        [1.35454965e+00, 5.71305580e+01, 1.05835907e+02, 3.55562469e+02],
        [1.11765953e+02, 0.00000000e+00, 5.10093353e+02, 2.71551788e+02],
        [4.19083710e+02, 1.09654984e+02, 5.10574158e+02, 4.08143280e+02],
        [4.22337952e+02, 3.46434479e+01, 4.43637878e+02, 7.50226746e+01],
        [1.86629211e+02, 8.86873932e+01, 5.12000000e+02, 4.20000000e+02],
        [4.40377092e+00, 0.00000000e+00, 4.34563416e+02, 2.34900024e+02],
        [3.29998718e+02, 7.87615347e+00, 5.12000000e+02, 4.15029205e+02],
        [0.00000000e+00, 2.70583487e+00, 2.15015991e+02, 3.48687561e+02],
        [5.44489574e+00, 1.02492569e+02, 4.87029053e+02, 4.20000000e+02],
        [8.71547997e-01, 9.26511536e+01, 1.47331131e+02, 4.20000000e+02],
        [8.03557739e+01, 0.00000000e+00, 3.90351624e+02, 3.04125671e+02],
        [2.60799988e+02, 4.36747932e+01, 4.63497742e+02, 2.59172333e+02],
        [1.53831983e+00, 1.78565216e+0

In [307]:
feature_example, bbox_example, (img_w_example, img_h_example) = openI_v_f[openI.id.iloc[2]]
feature_example.shape, bbox_example.shape, (img_w_example, img_h_example)

((36, 4), (36, 2048), (512, 512))

In [360]:
import pickle5 as pickle5

data_v_f = pickle5.load( open( "/home/alfirsafauzulh@student.ub.ac.id/BERTHop/transformed.pickle", "rb" ) )
data_v_f

{'a8ac1969.jpg': array([[ 1.2528754e+03, -9.4315662e+00,  7.5287018e+02, ...,
          4.5150883e+01,  4.5475006e+00,  2.1608978e+01],
        [-1.4496803e+01,  1.0180580e+01, -1.9979240e+01, ...,
          5.1132593e+00,  1.0336160e+01,  3.4518242e-01],
        [-6.0984697e+00, -7.2594452e+00,  1.0913544e+00, ...,
         -8.0199776e+00, -1.4643497e+01,  5.1947899e+00],
        ...,
        [-5.2379761e+02,  2.3171846e+02,  9.3460645e+02, ...,
          8.5185541e+02,  8.5473824e+01, -3.1537985e+02],
        [ 3.5902051e+02, -3.9196396e+02,  5.4213947e+01, ...,
          3.8620552e+01, -6.4647369e+00, -2.0015956e+02],
        [ 3.3194244e+01,  5.1776373e+02,  7.2235547e+02, ...,
          2.5826378e+00,  2.5056464e+02,  2.7982501e+02]], dtype=float32),
 '54d57e77.jpg': array([[ 4.01171899e+03, -3.61221581e+01,  5.46393188e+02, ...,
          1.06290970e+02,  7.15495300e+00,  3.11578369e+00],
        [-5.96624756e+00,  9.72837830e+00, -1.87318878e+01, ...,
          8.05239105e+00, -

In [362]:
# get all Index Name
index_name = data_xray.index.values

xray_v_f = {}
for name in index_name:
    xray_v_f[name] = data_v_f[name]

xray_v_f

{'auntminnie-a-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg': array([[ 5.73943750e+03, -4.53361694e+02,  1.00161121e+03, ...,
          4.91949310e+01,  1.88329620e+01,  5.56809998e+00],
        [-1.06974487e+01,  3.63324585e+01, -1.26057037e+02, ...,
         -6.24096298e+00,  2.25892277e+01, -3.96878052e+00],
        [ 1.49068283e+02,  1.07298584e+02, -1.61462097e+01, ...,
         -4.13150454e+00, -1.34687557e+01, -4.25308228e+00],
        ...,
        [-2.99960632e+02,  1.19930855e+02,  5.15795532e+02, ...,
          7.37459717e+02,  7.67609253e+01, -2.71423035e+02],
        [ 1.94910645e+02, -2.20784103e+02,  2.74222527e+01, ...,
          3.27003403e+01, -6.28549576e+00, -1.74579712e+02],
        [ 2.31390800e+01,  2.88924042e+02,  4.02231598e+02, ...,
         -3.64326477e-01,  2.12285461e+02,  2.38205719e+02]], dtype=float32),
 'auntminnie-b-2020_01_28_23_51_6665_2020_01_28_Vietnam_coronavirus.jpeg': array([[ 5.2882500e+03, -1.0003914e+02,  1.2916299e+03, ...,
    

In [366]:
assert len(list(xray_v_f.keys())) == len(data_xray.values), "Visual Features are inconsistent with openI dataset"

#### **Now We have download all data, models, and dependencies. We are good to go!!!**
**1. Change default arguments**

First, let's check it out!

In [367]:
# set working directory to the root of the project
import os

os.chdir('/home/alfirsafauzulh@student.ub.ac.id/BERTHop')

from param import args

args.__dict__

{'model': 'uniter',
 'train': 'train,nominival',
 'valid': 'minival',
 'test': None,
 'batch_size': 32,
 'optim': 'bert',
 'lr': 0.5,
 'epochs': 2,
 'dropout': 0.1,
 'seed': 9595,
 'max_seq_length': 128,
 'output': 'models/trained/',
 'fast': False,
 'tiny': False,
 'tqdm': True,
 'load_trained': None,
 'load_pretrained': '/home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA/models/pretrained/uniter-base.pt',
 'from_scratch': False,
 'mce_loss': False,
 'multiGPU': False,
 'num_workers': 0,
 'optimizer': 'bert'}

***1.1*** Let's overwrite some arguments***

In [368]:
args.batch_size = 32
args.epochs = 2
args.lr = 0.5
args.dropout = 0.1
# args.model = 'visualbert' # use visualbert
args.model = 'uniter' # use visualbert
# args.load_pretrained = '/home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA/models/pretrained/visualbert.th' #load pretrained visualbert model
args.load_pretrained = '/home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA/models/pretrained/uniter-base.pt' #load pretrained uniter-base model
args.max_seq_length = 128 #truncate or pad report lengths to 128 subwords

args.__dict__

{'model': 'uniter',
 'train': 'train,nominival',
 'valid': 'minival',
 'test': None,
 'batch_size': 32,
 'optim': 'bert',
 'lr': 0.5,
 'epochs': 2,
 'dropout': 0.1,
 'seed': 9595,
 'max_seq_length': 128,
 'output': 'models/trained/',
 'fast': False,
 'tiny': False,
 'tqdm': True,
 'load_trained': None,
 'load_pretrained': '/home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA/models/pretrained/uniter-base.pt',
 'from_scratch': False,
 'mce_loss': False,
 'multiGPU': False,
 'num_workers': 0,
 'optimizer': 'bert'}

#### **2. Create customized dataloader**

In [369]:
findings = list(openI.columns[1:-2])
findings

['Atelectasis',
 'Cardiomegaly',
 'Effusion',
 'Infiltration',
 'Mass',
 'Nodule',
 'Pneumonia',
 'Pneumothorax',
 'Consolidation',
 'Edema',
 'Emphysema',
 'Fibrosis',
 'Pleural_Thickening',
 'Hernia',
 'Normal']

In [371]:
new_findings = list(data_xray.columns[1:-2])
new_findings

['todo',
 'No Finding',
 'Pneumonia',
 'Pneumonia/Aspiration',
 'Pneumonia/Bacterial',
 'Pneumonia/Bacterial/Chlamydophila',
 'Pneumonia/Bacterial/E.Coli',
 'Pneumonia/Bacterial/Klebsiella',
 'Pneumonia/Bacterial/Legionella',
 'Pneumonia/Bacterial/Mycoplasma',
 'Pneumonia/Bacterial/Nocardia',
 'Pneumonia/Bacterial/Staphylococcus/MRSA',
 'Pneumonia/Bacterial/Streptococcus',
 'Pneumonia/Fungal/Aspergillosis',
 'Pneumonia/Fungal/Pneumocystis',
 'Pneumonia/Lipoid',
 'Pneumonia/Viral/COVID-19',
 'Pneumonia/Viral/Herpes ',
 'Pneumonia/Viral/Influenza',
 'Pneumonia/Viral/Influenza/H1N1',
 'Pneumonia/Viral/MERS-CoV',
 'Pneumonia/Viral/SARS',
 'Pneumonia/Viral/Varicella',
 'Tuberculosis',
 'Unknown']

In [372]:
from torch.utils.data import Dataset
from torch.utils.data.dataloader import DataLoader
import numpy as np
class OpenIDataset(Dataset):
  def __init__(self, df, vf, split, model = 'lxmert'):
    # train_test_split and prepare labels
    self.dataset = df[df['split'] == split]
    self.visual_features = vf
    self.id_list = self.dataset.id.tolist()
    self.report_list = self.dataset.TXT.tolist()
    self.findings_list = self.dataset.columns[1:-2]
    self.target_list = self.dataset[self.findings_list].to_numpy().astype(np.float32)
    self.model = model

  def __len__(self):
    return len(self.id_list)

  def __getitem__(self, item):
    cxr_id = self.id_list[item]
    target = self.target_list[item]
    boxes, feats, (img_w, img_h) = self.visual_features[cxr_id]
    report = self.report_list[item]
    if self.model == 'uniter':
      boxes = self._uniterBoxes(boxes)
    if self.model == 'lxmert':
      boxes[:, (0, 2)] /= img_w
      boxes[:, (1, 3)] /= img_h
    return cxr_id, feats, boxes, report, target

  def _uniterBoxes(self, boxes):#uniter requires a 7-dimensiom beside the regular 4-d bbox
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes = np.zeros((boxes.shape[0],7),dtype='float32')
    new_boxes[:,1] = boxes[:,0]
    new_boxes[:,0] = boxes[:,1]
    new_boxes[:,3] = boxes[:,2]
    new_boxes[:,2] = boxes[:,3]
    new_boxes[:,4] = new_boxes[:,3]-new_boxes[:,1] #w
    new_boxes[:,5] = new_boxes[:,2]-new_boxes[:,0] #h
    new_boxes[:,6]=new_boxes[:,4]*new_boxes[:,5] #area
    return new_boxes  

In [386]:
# training = OpenIDataset(df = openI, vf = openI_v_f,  split='train', model = args.model)
# testing = OpenIDataset(df = openI, vf = openI_v_f,  split='test', model = args.model)
training = OpenIDataset(df = data_xray, vf = xray_v_f,  split='train', model = args.model)
testing = OpenIDataset(df = data_xray, vf = xray_v_f,  split='test', model = args.model)

In [374]:
train_loader = DataLoader(training, batch_size=args.batch_size,shuffle=True, num_workers=0,drop_last=True, pin_memory=True)
test_loader = DataLoader(testing, batch_size=128,shuffle=False, num_workers=0,drop_last=False, pin_memory=True)

#### **3. Model, Optimizer, Loss Function, and Evaluation Function**

In [375]:
# chage working directory to the root of the project
os.chdir('/home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA')

from vqa_model import VQAModel

#init model
model = VQAModel(num_answers = len(findings), model = args.model)

In [376]:
#load pretrained weights using VISUALBERT
model.encoder.load(args.load_pretrained)

Load UNITER PreTrained Model from /home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA/models/pretrained/uniter-base.pt

Weights in loaded but not in model:
cls.predictions.bias
cls.predictions.decoder.weight
cls.predictions.transform.LayerNorm.bias
cls.predictions.transform.LayerNorm.weight
cls.predictions.transform.dense.bias
cls.predictions.transform.dense.weight
feat_regress.bias
feat_regress.net.0.bias
feat_regress.net.0.weight
feat_regress.net.2.bias
feat_regress.net.2.weight
feat_regress.weight
itm_output.bias
itm_output.weight
region_classifier.net.0.bias
region_classifier.net.0.weight
region_classifier.net.2.bias
region_classifier.net.2.weight
region_classifier.net.3.bias
region_classifier.net.3.weight
uniter.img_embeddings.mask_embedding.weight

Weights in model but not in loaded:



In [377]:
#send to GPU
model = model.cuda()


In [378]:
import torch
loss = torch.nn.BCEWithLogitsLoss()

In [379]:
from src.optimization import BertAdam

optim = BertAdam(list(model.parameters()),lr=args.lr,warmup=0.1,t_total=len(train_loader)*args.epochs)
# t_total denotes total training steps
# batch_per_epoch = len(train_loader)
# t_total = int(batch_per_epoch * args.epochs)

In [380]:
# Evaluation function, we will report the AUC and accuray of each finding
def eval(target, pred):
    acc_list = []
    for i, d in enumerate(findings[:-1]): #normal is excluded
        acc = np.mean(target[:,i] == (pred[:,i]>=0.5))
        print(i,d,acc)
        acc_list.append(acc)
    print('Averaged: '+str(np.average(acc_list)))

In [381]:
# activation = torch.nn.Sigmoid()
activation = torch.nn.Softmax(dim=1)

#### **4. HIT and RUN**

##### Using VISUALBERT Architecture

In [382]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))


  0%|          | 0/21 [00:00<?, ?it/s]

KeyError: 215

##### Using UNITER Architecture

In [125]:
from tqdm.notebook import tqdm

iter_wrapper = (lambda x: tqdm(x, total=len(train_loader))) if args.tqdm else (lambda x: x)
best_valid = 0
for epoch in range(args.epochs):
  epoch_loss = 0
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(train_loader)):
    model.train()
    optim.zero_grad()
    feats, boxes, target = feats.cuda(), boxes.cuda(), target.cuda()
    logit = model(feats, boxes, report)
    running_loss = loss(logit, target)
    running_loss = running_loss * logit.size(1)
    epoch_loss += running_loss
    running_loss.backward()
    torch.nn.utils.clip_grad_norm_(model.parameters(), 5.)
    optim.step()
  print("Epoch "+str(epoch)+": Training Loss: "+str(epoch_loss/len(train_loader)))
  print('Evaluation: ')
  model.eval()
  logit_list, target_list = [], []
  iter_wrapper = (lambda x: tqdm(x, total=len(test_loader)))
  for i, (cxr_id, feats, boxes, report, target) in iter_wrapper(enumerate(test_loader)):
    target_list.append(target)
    with torch.no_grad():
      feats, boxes = feats.cuda(), boxes.cuda()
      logit = model(feats, boxes, report)
      logit_list.append(activation(logit).cpu().numpy())

  eval(np.concatenate(target_list,axis = 0), np.concatenate(logit_list,axis = 0))

  0%|          | 0/91 [00:00<?, ?it/s]

/home/alfirsafauzulh@student.ub.ac.id/Transformers-VQA/src/optimization.py:142: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  ../torch/csrc/utils/python_arg_parser.cpp:1025.)
  next_m.mul_(beta1).add_(1 - beta1, grad)


Epoch 0: Training Loss: tensor(2447.3066, device='cuda:0', grad_fn=<DivBackward0>)
Evaluation: 


  0%|          | 0/7 [00:00<?, ?it/s]

0 Pneumonia/Viral/COVID-19 0.8989637305699482
1 Pneumonia 0.8963730569948186
2 Pneumonia/Viral/SARS 0.9455958549222798
3 Pneumonia/Fungal/Pneumocystis 0.9715025906735751
4 Pneumonia/Bacterial/Streptococcus 0.9961139896373057
5 No Finding 0.966321243523316
6 Pneumonia/Bacterial/Chlamydophila 0.9857512953367875
7 Pneumonia/Bacterial/E.Coli 0.9935233160621761
8 Pneumonia/Bacterial/Klebsiella 0.9857512953367875
9 Pneumonia/Bacterial/Legionella 0.9805699481865285
10 Unknown 0.9676165803108808
11 Pneumonia/Lipoid 0.9935233160621761
12 Pneumonia/Viral/Varicella 0.02072538860103627
13 Pneumonia/Bacterial 1.0
Averaged: 0.9001665433012584


  0%|          | 0/7 [00:00<?, ?it/s]